In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
import numpy as np

def build_autoencoder(input_dim, encoder_dims, bottleneck_dim):
    input_layer = Input(shape=(input_dim,))
    x = input_layer
    for dim in encoder_dims:
        x = Dense(dim, activation='relu')(x)
    bottleneck = Dense(bottleneck_dim, activation='relu')(x)
    for dim in reversed(encoder_dims):
        bottleneck = Dense(dim, activation='relu')(bottleneck)
    output_layer = Dense(input_dim, activation='sigmoid')(bottleneck)
    autoencoder = Model(inputs=input_layer, outputs=output_layer)
    return autoencoder

def train_autoencoder(autoencoder, x_train, x_test, epochs=50, batch_size=256):
    autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
    autoencoder.fit(x_train, x_train,
                    epochs=epochs,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(x_test, x_test))

# Load and preprocess the MNIST dataset
(x_train, _), (x_test, _) = mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

# Define AutoEncoders
input_dim = 784  # 28*28 pixels flattened
encoder_dims = [512, 256]  # Encoder and decoder dims (symmetric)
bottleneck_dim = 128  # Bottleneck dimension

# Build and train the first AutoEncoder
first_ae = build_autoencoder(input_dim, encoder_dims, bottleneck_dim)
train_autoencoder(first_ae, x_train, x_test)

# Extract features to train the next AutoEncoder
get_bottleneck_output = Model(inputs=first_ae.input, outputs=first_ae.layers[-3].output)
bottleneck_features = get_bottleneck_output.predict(x_train)

# Assuming similar steps are repeated for second and third AutoEncoders, with appropriate input adjustments

# Calculate the total number of parameters
total_params = first_ae.count_params()  # + second_ae.count_params() + third_ae.count_params() for the full chain
print(f"Total parameters in the network: {total_params}")

# NOTE: For simplicity, this script only fully implements and trains the first AutoEncoder.
# Extending this to epitaxially connect and train subsequent AutoEncoders would involve additional steps
# to manage the data flow and training process across the chain of AutoEncoders.


11490434/11490434 [==============================] - 0s 0us/step
Epoch 1/50
235/235 [==============================] - 13s 46ms/step - loss: 0.1960 - val_loss: 0.1210
Epoch 2/50
235/235 [==============================] - 10s 44ms/step - loss: 0.1083 - val_loss: 0.0978
Epoch 3/50
235/235 [==============================] - 10s 41ms/step - loss: 0.0946 - val_loss: 0.0896
Epoch 4/50
235/235 [==============================] - 11s 45ms/step - loss: 0.0887 - val_loss: 0.0853
Epoch 5/50
235/235 [==============================] - 11s 45ms/step - loss: 0.0848 - val_loss: 0.0827
Epoch 6/50
235/235 [==============================] - 11s 45ms/step - loss: 0.0823 - val_loss: 0.0805
Epoch 7/50
235/235 [==============================] - 10s 45ms/step - loss: 0.0803 - val_loss: 0.0791
Epoch 8/50
235/235 [==============================] - 9s 40ms/step - loss: 0.0790 - val_loss: 0.0779
Epoch 9/50
235/235 [==============================] - 11s 45ms/step - loss: 0.0777 - val_loss: 0.0775
Epoch 10/50
235/23